In [6]:
import rasterio as rio
import geopandas as gpd
import fiona
import os
import pickle
# enable kml file reading for geopandas
from shapely.geometry import box, Polygon
import pandas as pd
import numpy as np
import leafmap.foliumap as leafmap # use folium backend
import rioxarray as rioxr
import matplotlib.pyplot as plt
from typing import List, Union, Dict
from pathlib import Path
import itertools
import glob
pd.options.display.max_columns = 50

In [7]:
import osmnx

In [8]:
from src.config import *
from src.io.utils_io import load_maxar_items, load_rs_path_toy, load_roi_town, check_dir, make_path
from src.utils import create_bbox, get_maxar_items_on_roi
from src.rs_processing.processing import crop_rs_on_shape
from src.stac.utils_stac import get_nearest_match_geometry_view

In [37]:
town_test = "Talat_Nyaaqoub"
date_event = '2023-09-09'
type_item="visual"
buffer_size=500
building_root_path = os.path.join(external_dir_path, "building_footprint")


In [10]:
maxar_items = load_maxar_items(type_item=type_item)

In [38]:
town = load_roi_town(buffer_size=buffer_size)

In [39]:
town = town.to_crs(4326)

### OSMNX

In [40]:
building = osmnx.features_from_polygon(town[town.town_name==town_test].geometry.item(), tags={"building":True})

In [41]:
building.explore()

In [47]:
building = building.to_crs(CRS)

In [49]:
building = building[["geometry"]]
building["town_name"] = town_test

/home/rustt/miniforge3/envs/damage_detection/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [50]:
building.to_file(make_path("building_osm.shp", building_root_path, town_test))

/tmp/ipykernel_267580/4278592986.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  building.to_file(make_path("building_osm.shp", building_root_path, town_test))


### COOP

In [30]:
coop_path = os.path.join(building_root_path, "building_COOP", "MAR.parquet")
coop = gpd.read_parquet(coop_path)

In [32]:
coop = coop.to_crs(4326)

In [42]:
building_coop = gpd.overlay(coop, town[town.town_name==town_test], how="intersection")

In [43]:
building_coop.shape

(700, 7)

In [44]:
m = building_coop.explore()
town[town.town_name==town_test].explore(m=m)

In [51]:
building_coop = building_coop.to_crs(CRS)

In [52]:
building_coop = building_coop[["geometry"]]
building_coop["town_name"] = town_test

In [54]:
building_coop.to_file(make_path("building_coop.shp", building_root_path, town_test))